<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [1]:
!pip install -r requirements.txt --user

  Cloning git://github.com/dvdpedros/pyfolio (to revision fix-extreme-drawdown) to c:\users\adega\appdata\local\temp\pip-req-build-h9um5_fq
  Running command git checkout -b fix-extreme-drawdown --track origin/fix-extreme-drawdown
  Branch 'fix-extreme-drawdown' set up to track remote branch 'fix-extreme-drawdown' from 'origin'.
  Switched to a new branch 'fix-extreme-drawdown'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

  Created wheel for empirical: filename=empirical-0.2-py3-none-any.whl size=50204 sha256=4411bbdfb024a747698bb0c04306ad98d1855e983a2ab7846c4ec16f5662f9cb
  Stored in directory: c:\users\adega\appdata\local\pip\cache\wheels\33\ac\24\c5d20fe896fce99f3f4b8b31aa9d09ff6b36610a17ebbd90a9
Successfully built empirical


<a id='1.2'></a>
## 2.2. Import Packages

In [2]:
%load_ext autoreload
%autoreload 
import pandas as pd
from config import config
from dataset.download_dataset.cryptodownloader_binance import CryptoDownloader_binance
from preprocessing.preprocessors import FeatureEngineer
from preprocessing.data import data_split
from env.env_custom import CustomTradingEnv
from model.models import DRLAgent
from trade.backtest import BackTest
import warnings
warnings.filterwarnings("ignore")

# from trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


C:\Users\adega\AppData\Roaming\Python\Python38\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.3'></a>
## 2.3 Create Folders

In [3]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [4]:
data_downloader = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.MULTIPLE_TICKER_8, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

Downloaded https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1h/BTCUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/ETHUSDT/1h/ETHUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/BNBUSDT/1h/BNBUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/ADAUSDT/1h/ADAUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1h/XRPUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/DOGEUSDT/1h/DOGEUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/LINKUSDT/1h/LINKUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/LTCUSDT/1h/LTCUSDT-1h-2020-01.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1h/BTCUSDT-1h-2020-02.zip
Downloaded https://data.binance.vision/data/spot/monthly/klines/ETHUSDT/1h/ETHUSDT-1h-2020-02.zip
Downloaded https

# Part 4: Preprocess Data

In [5]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = True)

df = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added user defined features


## Add covariance matrix as states

In [6]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
        

In [7]:
df

,date,open,high,low,close,volume,tic,psar,ui,atr,...,dpo_diff,kst_diff,aroon_up_diff,aroon_down_diff,ppo_diff,open_diff,close_diff,high_diff,low_diff,cov_list
0,2020-01-11 12:00:00,0.036630,0.036690,0.036410,0.036670,2.592805e+06,ada,0.0,0.797276,0.000370,...,9.230769,-0.075530,-0.062500,1.000000,-0.129260,-0.002179,0.001092,-0.001633,-0.000549,"[[4.023392640830573e-05, 2.823734010252163e-05..."
1,2020-01-11 12:00:00,14.868600,14.920900,14.820000,14.900300,1.779574e+04,bnb,0.0,1.110773,0.136440,...,1.041763,-0.063189,-0.076923,1.000000,-0.062154,-0.000726,0.002267,0.001463,0.000567,"[[4.023392640830573e-05, 2.823734010252163e-05..."
2,2020-01-11 12:00:00,8045.890000,8068.000000,8018.410000,8061.950000,1.788465e+03,btc,0.0,1.267836,83.737412,...,0.390335,-0.073836,-0.062500,1.000000,-0.118173,-0.001011,0.001989,-0.000775,0.001050,"[[4.023392640830573e-05, 2.823734010252163e-05..."
3,2020-01-11 12:00:00,0.002179,0.002184,0.002174,0.002174,1.427073e+06,doge,0.0,1.268733,0.000056,...,0.966287,-0.104201,-0.062500,0.000000,-0.062768,-0.000917,-0.002570,-0.001874,0.001382,"[[4.023392640830573e-05, 2.823734010252163e-05..."
4,2020-01-11 12:00:00,142.630000,142.940000,142.100000,142.710000,5.180679e+03,eth,0.0,1.132186,1.343489,...,1.124724,-0.063166,-0.076923,1.000000,-0.095866,-0.001889,0.000561,-0.002721,-0.001826,"[[4.023392640830573e-05, 2.823734010252163e-05..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85531,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,0.0,1.529060,0.000638,...,-0.141017,-0.098274,-0.250000,-0.076923,-0.261475,-0.006363,0.011816,0.000481,0.011784,"[[0.0001165532169555054, 6.743961275719393e-05..."
85532,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,0.0,1.461852,24.323171,...,-0.041898,0.224770,-0.043478,-0.076923,0.059607,-0.013487,0.008088,-0.006362,-0.000904,"[[0.0001165532169555054, 6.743961275719393e-05..."
85533,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.0,1.670446,0.441052,...,-0.121672,-2.019205,0.041667,-0.083333,0.629285,0.000497,0.029755,0.020620,-0.000259,"[[0.0001165532169555054, 6.743961275719393e-05..."
85534,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.0,2.374702,2.674103,...,-0.185957,-0.238845,-0.333333,-0.076923,3.732029,-0.003533,0.010636,0.003198,-0.000719,"[[0.0001165532169555054, 6.743961275719393e-05..."


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-01-01 to 2020-12-31

In [8]:
train = data_split(df, '2020-01-01','2020-12-31')

## Environment for Portfolio Allocation


In [9]:
stock_dimension = len(config.SINGLE_TICKER)
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
features = config.TECHNICAL_INDICATORS_LIST + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]

env_kwargs_singleticker = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": config.SINGLE_TICKER,
    "all_tickers": train.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.1,
    "discrete_actionspace":True
    
}

e_train_gym_singleticker = CustomTradingEnv(df = train, **env_kwargs_singleticker)

env_train_discreteactionspace, _ = e_train_gym_singleticker.get_sb_env()

Stock Dimension: 1, State Space: 21


In [10]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
features = config.TECHNICAL_INDICATORS_LIST + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]

env_kwargs_multipleticker = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": train.tic.unique(),
    "all_tickers": train.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}

e_train_gym_multipleticker = CustomTradingEnv(df = train, **env_kwargs_multipleticker)

env_train_continousactionspace, _ = e_train_gym_multipleticker.get_sb_env()

Stock Dimension: 8, State Space: 8


<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [11]:
# initialize
agent_singleticker = DRLAgent(env = env_train_discreteactionspace)
agent_multipleticker = DRLAgent(env = env_train_continousactionspace)

### Model 1: **DQN**

In [12]:
model_dqn = agent_singleticker.get_model(model_name="dqn",model_kwargs=config.DQN_PARAMS)

{'buffer_size': 10000, 'learning_rate': 0.00025, 'exploration_initial_eps': 1.0, 'exploration_fraction': 0.999985, 'exploration_final_eps': 0.02, 'gamma': 0.99, 'batch_size': 100}
Using cuda device


In [13]:
trained_dqn = agent_singleticker.train_model(model=model_dqn, 
                                tb_log_name='dqn',
                                total_timesteps=10000)

Logging to ./tensorboard_log/dqn\dqn_1


### Model 2: **A2C**


In [14]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent_multipleticker.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cuda device


In [15]:
trained_a2c = agent_multipleticker.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=10000)

Logging to ./tensorboard_log/a2c\a2c_1
------------------------------------
| time/                 |          |
|    fps                | 18       |
|    iterations         | 100      |
|    time_elapsed       | 26       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -11.4    |
|    explained_variance | 0.21     |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | 8.59     |
|    std                | 1        |
|    value_loss         | 0.841    |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 19       |
|    iterations         | 200      |
|    time_elapsed       | 51       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -11.4    |
|    explained_variance | -5.07    |
|    learning_rate      | 0.0002   |
|    n_updates          | 199      |

### Model 3: **PPO**

In [16]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [17]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [18]:
trade_dqn = data_split(df,config.START_TEST_DATE, config.END_DATE)
e_trade_gym_dqn = CustomTradingEnv(df = trade_dqn, **env_kwargs_singleticker)

In [19]:
trade_a2c = data_split(df,config.START_TEST_DATE, config.END_DATE)
e_trade_gym_a2c = CustomTradingEnv(df = trade_a2c, **env_kwargs_multipleticker)

In [20]:
trade_dqn.shape

(17280, 47)

In [21]:
trade_a2c.shape

(17280, 47)

In [22]:
trade_dqn

,date,open,high,low,close,volume,tic,psar,ui,atr,...,dpo_diff,kst_diff,aroon_up_diff,aroon_down_diff,ppo_diff,open_diff,close_diff,high_diff,low_diff,cov_list
0,2021-01-01 00:00:00,0.181340,0.181460,0.178310,0.180510,1.919492e+07,ada,0.0,3.174316,0.003072,...,-0.137262,-0.578558,-0.083333,-0.062500,-0.082879,0.008789,-0.004577,-0.005971,-0.000952,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,37.359600,37.442300,36.963600,37.376400,9.511383e+04,bnb,1.0,2.328882,0.404216,...,-0.902435,-0.135416,10.000000,-0.062500,-0.190181,0.003985,0.000471,0.000513,-0.002986,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,28923.630000,29031.340000,28690.170000,28995.130000,2.311811e+03,btc,0.0,1.505422,378.624288,...,0.408664,0.050703,-0.111111,-0.062500,-0.002647,-0.006089,0.002472,-0.002714,-0.003121,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,0.004672,0.004701,0.004601,0.004679,2.768207e+07,doge,0.0,2.016111,0.000074,...,-0.150362,0.078663,-0.083333,-0.062500,0.074471,-0.000193,0.001691,-0.000829,-0.006263,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,736.420000,739.000000,729.330000,734.070000,2.793270e+04,eth,0.0,1.998452,9.626878,...,0.175328,0.123737,10.000000,-0.055556,2.704563,-0.005456,-0.003191,-0.003640,-0.003416,"[[0.00023491493831790994, 7.732551281244585e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,0.0,1.529060,0.000638,...,-0.141017,-0.098274,-0.250000,-0.076923,-0.261475,-0.006363,0.011816,0.000481,0.011784,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,0.0,1.461852,24.323171,...,-0.041898,0.224770,-0.043478,-0.076923,0.059607,-0.013487,0.008088,-0.006362,-0.000904,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.0,1.670446,0.441052,...,-0.121672,-2.019205,0.041667,-0.083333,0.629285,0.000497,0.029755,0.020620,-0.000259,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.0,2.374702,2.674103,...,-0.185957,-0.238845,-0.333333,-0.076923,3.732029,-0.003533,0.010636,0.003198,-0.000719,"[[0.0001165532169555054, 6.743961275719393e-05..."


In [23]:
trade_a2c

,date,open,high,low,close,volume,tic,psar,ui,atr,...,dpo_diff,kst_diff,aroon_up_diff,aroon_down_diff,ppo_diff,open_diff,close_diff,high_diff,low_diff,cov_list
0,2021-01-01 00:00:00,0.181340,0.181460,0.178310,0.180510,1.919492e+07,ada,0.0,3.174316,0.003072,...,-0.137262,-0.578558,-0.083333,-0.062500,-0.082879,0.008789,-0.004577,-0.005971,-0.000952,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,37.359600,37.442300,36.963600,37.376400,9.511383e+04,bnb,1.0,2.328882,0.404216,...,-0.902435,-0.135416,10.000000,-0.062500,-0.190181,0.003985,0.000471,0.000513,-0.002986,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,28923.630000,29031.340000,28690.170000,28995.130000,2.311811e+03,btc,0.0,1.505422,378.624288,...,0.408664,0.050703,-0.111111,-0.062500,-0.002647,-0.006089,0.002472,-0.002714,-0.003121,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,0.004672,0.004701,0.004601,0.004679,2.768207e+07,doge,0.0,2.016111,0.000074,...,-0.150362,0.078663,-0.083333,-0.062500,0.074471,-0.000193,0.001691,-0.000829,-0.006263,"[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,736.420000,739.000000,729.330000,734.070000,2.793270e+04,eth,0.0,1.998452,9.626878,...,0.175328,0.123737,10.000000,-0.055556,2.704563,-0.005456,-0.003191,-0.003640,-0.003416,"[[0.00023491493831790994, 7.732551281244585e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,0.0,1.529060,0.000638,...,-0.141017,-0.098274,-0.250000,-0.076923,-0.261475,-0.006363,0.011816,0.000481,0.011784,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,0.0,1.461852,24.323171,...,-0.041898,0.224770,-0.043478,-0.076923,0.059607,-0.013487,0.008088,-0.006362,-0.000904,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.0,1.670446,0.441052,...,-0.121672,-2.019205,0.041667,-0.083333,0.629285,0.000497,0.029755,0.020620,-0.000259,"[[0.0001165532169555054, 6.743961275719393e-05..."
2159,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.0,2.374702,2.674103,...,-0.185957,-0.238845,-0.333333,-0.076923,3.732029,-0.003533,0.010636,0.003198,-0.000719,"[[0.0001165532169555054, 6.743961275719393e-05..."


In [24]:
allocations_dqn, transactions_dqn, allocation_values_dqn = agent_singleticker.DRL_prediction(model=trained_dqn,
                        environment = e_trade_gym_dqn)

hit end!


In [25]:
allocations_dqn

,cash,btc,date
0,11266.067,31.0,2021-01-01 01:00:00
0,11266.067,31.0,2021-01-01 02:00:00
0,11266.067,31.0,2021-01-01 03:00:00
0,11266.067,31.0,2021-01-01 04:00:00
0,11266.067,31.0,2021-01-01 05:00:00
...,...,...,...
0,11266.067,31.0,2021-03-31 18:00:00
0,11266.067,31.0,2021-03-31 19:00:00
0,11266.067,31.0,2021-03-31 20:00:00
0,11266.067,31.0,2021-03-31 21:00:00


In [26]:
transactions_dqn

,date,operation,tic,amount,price
0,2021-01-01,buy,btc,31.0,28995.13


In [27]:
allocation_values_dqn

,cash,btc,date
0,11266.067,911709.69,2021-01-01 01:00:00
0,11266.067,905034.15,2021-01-01 02:00:00
0,11266.067,907630.40,2021-01-01 03:00:00
0,11266.067,905829.61,2021-01-01 04:00:00
0,11266.067,904797.31,2021-01-01 05:00:00
...,...,...,...
0,11266.067,1838282.95,2021-03-31 18:00:00
0,11266.067,1815146.72,2021-03-31 19:00:00
0,11266.067,1826972.60,2021-03-31 20:00:00
0,11266.067,1827155.50,2021-03-31 21:00:00


In [28]:
allocations_a2c, transactions_a2c, allocation_values_a2c = agent_multipleticker.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym_a2c)

hit end!


In [29]:
allocations_a2c

,cash,ada,bnb,btc,doge,eth,link,ltc,xrp,date
0,997285.612367,0.000000,64.893824,0.0,18.472964,0.000000,21.491222,0.000000,87.906253,2021-01-01 01:00:00
0,16.988847,0.000000,108.982360,33.0,0.000000,20.000000,19.067644,2.000000,110.375443,2021-01-01 02:00:00
0,65.213237,0.000000,99.938168,33.0,100.000000,20.000000,34.346244,3.000000,10.375443,2021-01-01 03:00:00
0,916307.555731,0.000000,67.828207,0.0,65.286484,56.395803,104.425982,103.000000,0.000000,2021-01-01 04:00:00
0,872595.277201,0.000000,118.991863,0.0,126.997486,100.815679,58.550695,170.887080,0.000000,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,19033.950436,100.000000,0.000000,0.0,0.000000,17.000000,327.158289,5230.511399,343.473485,2021-03-31 18:00:00
0,180.441581,125.230676,61.000000,0.0,0.000000,11.940105,427.158289,5264.511399,243.473485,2021-03-31 19:00:00
0,10431.251906,189.079374,61.000000,0.0,0.000000,0.000000,522.129302,5312.788434,324.086192,2021-03-31 20:00:00
0,3434.725613,89.079374,25.387217,0.0,0.000000,10.000000,495.547490,5306.739945,386.966971,2021-03-31 21:00:00


In [30]:
transactions_a2c

,date,operation,tic,amount,price
0,2021-01-01 00:00:00,buy,bnb,64.893824,37.376400
0,2021-01-01 00:00:00,buy,doge,18.472964,0.004679
0,2021-01-01 00:00:00,buy,link,21.491222,11.281200
0,2021-01-01 00:00:00,buy,xrp,87.906253,0.221620
0,2021-01-01 01:00:00,buy,bnb,44.088537,37.613400
...,...,...,...,...,...
0,2021-03-31 21:00:00,buy,doge,13.053598,0.053500
0,2021-03-31 21:00:00,sell,eth,10.000000,1929.990000
0,2021-03-31 21:00:00,buy,link,36.226836,28.572800
0,2021-03-31 21:00:00,buy,ltc,92.000000,195.290000


In [31]:
allocation_values_a2c

,cash,ada,bnb,btc,doge,eth,link,ltc,xrp,date
0,997285.612367,0.000000,2440.877354,0.00,0.087497,0.000000,246.579537,0.000000e+00,19.675178,2021-01-01 01:00:00
0,16.988847,0.000000,4136.970399,963423.45,0.000000,14881.200000,220.751830,2.666600e+02,24.681053,2021-01-01 02:00:00
0,65.213237,0.000000,3790.155036,966187.20,0.497810,14896.400000,395.002411,3.949800e+02,2.358338,2021-01-01 03:00:00
0,916307.555731,0.000000,2568.667782,0.00,0.319499,41862.040679,1196.460685,1.347961e+04,0.000000,2021-01-01 04:00:00
0,872595.277201,0.000000,4487.528229,0.00,0.627609,74669.132529,670.001453,2.236570e+04,0.000000,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,19033.950436,118.784000,0.000000,0.00,0.000000,32159.410000,9248.175935,1.014510e+06,189.827491,2021-03-31 18:00:00
0,180.441581,148.241813,18351.111900,0.00,0.000000,22693.721919,11942.491433,1.016314e+06,133.985894,2021-03-31 19:00:00
0,10431.251906,224.932605,18315.506200,0.00,0.000000,0.000000,14852.490116,1.039500e+06,182.930451,2021-03-31 20:00:00
0,3434.725613,106.483702,7618.107271,0.00,0.000000,19299.900000,14159.179316,1.036353e+06,216.906596,2021-03-31 21:00:00


<a id='6'></a>
# Part 7: Backtest

In [32]:
%load_ext autoreload
%autoreload 
bat_dqn = BackTest(trained_dqn, e_trade_gym_dqn)
bat_dqn.create_summary(allocations_dqn, allocation_values_dqn, transactions_dqn)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Start date,2021-01-01
End date,2021-03-31
Total months,4
,Backtest
Annual return,583.229%
Cumulative returns,98.637%
Annual volatility,77.307%
Sharpe ratio,2.90
Calmar ratio,23.37
Stability,0.78
Max drawdown,-24.951%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,24.95,2021-01-08,2021-01-27,2021-02-08,22
1,21.24,2021-02-21,2021-02-28,2021-03-11,14
2,16.08,2021-03-13,2021-03-25,NaT,NaN
3,3.45,2021-02-09,2021-02-10,2021-02-11,3
4,3.03,2021-01-03,2021-01-04,2021-01-05,2


Stress Events,mean,min,max
New Normal,0.89%,-12.89%,19.35%


Top 10 long positions of all time,max
btc,0.27%


Top 10 short positions of all time,max


Top 10 positions of all time,max
btc,0.27%
index,0.00%


In [33]:
bat_dqn.plot_return_against_hold(allocation_values_dqn)

In [34]:
%load_ext autoreload
%autoreload 
bat_a2c = BackTest(trained_a2c, e_trade_gym_a2c)
bat_a2c.create_summary(allocations_a2c, allocation_values_a2c, transactions_a2c)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Start date,2021-01-01
End date,2021-03-31
Total months,4
,Backtest
Annual return,39.202%
Cumulative returns,12.539%
Annual volatility,94.218%
Sharpe ratio,0.82
Calmar ratio,1.07
Stability,0.05
Max drawdown,-36.534%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,36.53,2021-01-06,2021-01-27,2021-02-13,28
1,34.00,2021-02-17,2021-03-25,NaT,NaN
2,8.22,2021-02-13,2021-02-15,2021-02-17,3
3,NaN,NaT,NaT,NaT,NaN
4,NaN,NaT,NaT,NaT,NaN


Stress Events,mean,min,max
New Normal,0.31%,-17.57%,19.36%


Top 10 long positions of all time,max
ltc,99.47%
doge,10.91%
link,10.62%
bnb,4.92%
eth,3.77%
xrp,3.56%
ada,2.68%
btc,0.03%


Top 10 short positions of all time,max


Top 10 positions of all time,max
ltc,99.47%
doge,10.91%
link,10.62%
bnb,4.92%
eth,3.77%
xrp,3.56%
ada,2.68%
btc,0.03%
index,0.00%


In [35]:
bat_a2c.plot_return_against_hold(allocation_values_a2c)